# Blood-Brain Barrier Permeability Analysis

This notebook provides an interactive analysis of drug BBB permeability using molecular descriptors.

In [ ]:
# Import required libraries
import sys
import os
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from rdkit import Chem
from rdkit.Chem import Descriptors, Draw
from rdkit.ML.Descriptors import MoleculeDescriptors
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Set up plotting
plt.style.use('seaborn-v0_8')
%matplotlib inline

## Load and Explore Data

In [ ]:
# Load the dataset
data_path = '../data/BBB_datasets.csv'
if os.path.exists(data_path):
    data_drug = pd.read_csv(data_path, encoding='latin-1')
    print(f"Dataset loaded: {data_drug.shape}")
    print(f"Columns: {data_drug.columns.tolist()}")
    print("\nFirst few rows:")
    display(data_drug.head())
else:
    print("Dataset not found. Please place BBB_datasets.csv in the data/ directory.")
    # Use sample data for demonstration
    data_drug = pd.read_csv('../data/sample_data.csv')
    print("Using sample data for demonstration.")
    display(data_drug.head())

## Molecular Descriptor Calculation

In [ ]:
# Calculate molecular descriptors
mol_descriptors = []
valid_indices = []

print("Calculating molecular descriptors...")
for i, smile in enumerate(data_drug['SMILES']):
    mol = Chem.MolFromSmiles(smile)
    if mol is not None:
        try:
            calc = MoleculeDescriptors.MolecularDescriptorCalculator([x[0] for x in Descriptors._descList])
            vector = calc.CalcDescriptors(mol)
            mol_descriptors.append(vector)
            valid_indices.append(i)
        except Exception as e:
            print(f"Error processing SMILES {i}: {smile}")
    else:
        print(f"Invalid SMILES at index {i}: {smile}")

print(f"Successfully processed {len(mol_descriptors)} molecules")

In [ ]:
# Create DataFrame with descriptors
cols_mols = [x[0] for x in Descriptors._descList]
desc_df = pd.DataFrame(mol_descriptors, columns=cols_mols)

# Filter original data to match valid indices
data_drug_filtered = data_drug.iloc[valid_indices].reset_index(drop=True)

print(f"Descriptor DataFrame shape: {desc_df.shape}")
print(f"Filtered data shape: {data_drug_filtered.shape}")

## Exploratory Data Analysis

In [ ]:
# Check for missing values
missing_values = desc_df.isna().sum()
print(f"Features with missing values: {missing_values[missing_values > 0].shape[0]}")

# Remove rows with missing values
desc_df_clean = desc_df.dropna().reset_index(drop=True)
data_drug_clean = data_drug_filtered.iloc[desc_df_clean.index].reset_index(drop=True)

print(f"Clean dataset shape: {desc_df_clean.shape}")
print(f"Class distribution:")
print(data_drug_clean['Class'].value_counts())

In [ ]:
# Basic statistics
desc_df_clean.describe()

## Principal Component Analysis

In [ ]:
# Standardize the data
scaler = StandardScaler()
desc_df_scaled = scaler.fit_transform(desc_df_clean)

# Perform PCA
pca = PCA(n_components=2)
desc_df_pca = pca.fit_transform(desc_df_scaled)

print(f"Explained variance ratio: {pca.explained_variance_ratio_}")
print(f"Total explained variance: {pca.explained_variance_ratio_.sum():.3f}")

In [ ]:
# Create PCA plot
plt.figure(figsize=(10, 6))

# Map class values to colors
color_map = {'BBB+': 'red', 'BBB-': 'blue'}
colors = [color_map.get(cls, 'gray') for cls in data_drug_clean['Class']]

scatter = plt.scatter(desc_df_pca[:, 0], desc_df_pca[:, 1], c=colors, alpha=0.7)
plt.xlabel(f'Principal Component 1 ({pca.explained_variance_ratio_[0]:.1%} variance)')
plt.ylabel(f'Principal Component 2 ({pca.explained_variance_ratio_[1]:.1%} variance)')
plt.title('PCA of Drug Descriptors')

# Create legend
from matplotlib.patches import Patch
legend_elements = [Patch(facecolor=color, label=label) for label, color in color_map.items()]
plt.legend(handles=legend_elements, title='BBB Permeability')

plt.tight_layout()
plt.show()

## Feature Analysis

In [ ]:
# Analyze key molecular descriptors
key_descriptors = ['MolWt', 'LogP', 'NumHDonors', 'NumHAcceptors', 'TPSA', 'NumRotatableBonds']

# Create subplots
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.ravel()

for i, desc in enumerate(key_descriptors):
    if desc in desc_df_clean.columns:
        # Create box plot
        data_for_plot = []
        labels = []
        
        for class_val in ['BBB+', 'BBB-']:
            mask = data_drug_clean['Class'] == class_val
            data_for_plot.append(desc_df_clean.loc[mask, desc])
            labels.append(class_val)
        
        axes[i].boxplot(data_for_plot, labels=labels)
        axes[i].set_title(f'{desc} Distribution')
        axes[i].set_ylabel(desc)
    else:
        axes[i].text(0.5, 0.5, f'{desc} not found', ha='center', va='center')
        axes[i].set_title(f'{desc} (Not Available)')

plt.tight_layout()
plt.show()

## Machine Learning Model Implementation


In [ ]:
# Import ML libraries
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, roc_curve
from sklearn.preprocessing import LabelEncoder
import joblib
import os

# Import our custom ML module
sys.path.append('../src')
from ml_models import BBBPredictor


In [ ]:
# Prepare data for machine learning
X = desc_df_scaled  # Already standardized features
y = data_drug_clean['Class'].values  # BBB permeability labels

# Initialize the BBB predictor
predictor = BBBPredictor(random_state=42)

# Prepare data
X_train, X_test, y_train, y_test = predictor.prepare_data(
    X, y, feature_names=desc_df_clean.columns.tolist()
)


In [ ]:
# Train models
results = predictor.train_models(X_train, y_train)


In [ ]:
# Evaluate models on test data
evaluation_results = predictor.evaluate_models(X_test, y_test)


In [ ]:
# Feature importance analysis
predictor.plot_feature_importance(top_n=20)


In [ ]:
# Model comparison
predictor.plot_model_comparison()


In [ ]:
# Confusion matrix for best model
best_model_name = max(predictor.results.keys(), key=lambda x: predictor.results[x].get('accuracy', 0))
y_pred_best = predictor.results[best_model_name]['predictions']

plt.figure(figsize=(8, 6))
cm = confusion_matrix(y_test, y_pred_best)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=predictor.label_encoder.classes_, 
            yticklabels=predictor.label_encoder.classes_)
plt.title(f'Confusion Matrix - {best_model_name}')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.tight_layout()
plt.show()


In [ ]:
# ROC Curve for best model
best_model_name = max(predictor.results.keys(), key=lambda x: predictor.results[x].get('accuracy', 0))
probabilities = predictor.results[best_model_name].get('probabilities')

if probabilities is not None:
    plt.figure(figsize=(8, 6))
    fpr, tpr, _ = roc_curve(y_test, probabilities)
    auc_score = roc_auc_score(y_test, probabilities)
    
    plt.plot(fpr, tpr, label=f'{best_model_name} (AUC = {auc_score:.3f})')
    plt.plot([0, 1], [0, 1], 'k--', label='Random')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend()
    plt.tight_layout()
    plt.show()
else:
    print(f"Model {best_model_name} does not support probability predictions.")


In [ ]:
# Save models
predictor.save_models('../results/models')

print(f"\nBest performing model: {best_model_name}")
print(f"Best accuracy: {predictor.results[best_model_name]['accuracy']:.3f}")


## Model Prediction Example

Let's demonstrate how to use the trained model to predict BBB permeability for new molecules.


In [ ]:
# Example: Predict BBB permeability for new molecules
new_smiles = [
    'CC(=O)OC1=CC=CC=C1C(=O)O',  # Aspirin
    'CC1=CC=C(C=C1)C2=CC(=O)C3=C(O2)C(=O)C4=C(C3=O)C=CC=C4O',  # Complex molecule
    'CN1C=NC2=C1C(=O)N(C(=O)N2C)C'  # Caffeine (known BBB+)
]

# Calculate descriptors for new molecules
new_descriptors = []
valid_smiles = []

for smile in new_smiles:
    mol = Chem.MolFromSmiles(smile)
    if mol is not None:
        try:
            calc = MoleculeDescriptors.MolecularDescriptorCalculator([x[0] for x in Descriptors._descList])
            vector = calc.CalcDescriptors(mol)
            new_descriptors.append(vector)
            valid_smiles.append(smile)
        except Exception as e:
            print(f"Error processing SMILES: {smile}")
    else:
        print(f"Invalid SMILES: {smile}")

if new_descriptors:
    # Create DataFrame and clean
    new_desc_df = pd.DataFrame(new_descriptors, columns=desc_df_clean.columns)
    new_desc_df = new_desc_df.dropna()
    
    # Make predictions
    predictions = predictor.predict(new_desc_df.values)
    probabilities = predictor.predict_proba(new_desc_df.values)
    
    # Display results
    results_df = pd.DataFrame({
        'SMILES': valid_smiles[:len(predictions)],
        'Predicted_Class': predictions,
        'BBB+_Probability': probabilities[:, 1],
        'BBB-_Probability': probabilities[:, 0]
    })
    
    print("Prediction Results:")
    display(results_df)
else:
    print("No valid molecules to predict.")


## Next Steps

This analysis provides the foundation for machine learning model development:

1. **Feature Selection**: Identify the most important descriptors
2. **Model Training**: Implement classification algorithms (Random Forest, SVM, etc.)
3. **Model Evaluation**: Cross-validation and performance metrics
4. **Feature Importance**: Analyze which descriptors are most predictive